In [4]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY") 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7, google_api_key=api_key) 

In [ ]:
from langchain.prompts import PromptTemplate

def is_coding(query):
    
    prompt = PromptTemplate.from_template(
        "Is the following question related to programming or software development? Reply with 'Yes' or 'No Strictly'.\n\nQuestion: {query}"
    )

    response = llm.invoke(prompt.format(query=query))
    response_text = response.content.strip().lower()  

    return response_text == "yes"

In [ ]:
import requests

def fetch_stackoverflow(query):
    url = f"https://api.stackexchange.com/2.3/search/advanced?order=desc&sort=relevance&q={query}&site=stackoverflow"
    response = requests.get(url)
    data = response.json()

    results = []
    for item in data.get("items", []):
        title = item.get("title", "No title available")
        link = item.get("link", "")
        answer_id = item.get("accepted_answer_id", "No answer ID")
        
        results.append(f"Title: {title}\nLink: {link}\nAnswer ID: {answer_id}")
    
    return results


In [ ]:
query = "How to fix ndk in flutter"

if is_coding(query):
    results = fetch_stackoverflow(query)
    links = [res.split("\n")[1].replace("Link: ", "") for res in results[:3]] if results else []

    if links:
        print("Stack Overflow Links:", links)
    else:
        print("No relevant answers found.")
else:
    print("This does not seem to be a coding-related question.")


Stack Overflow Links: ['https://stackoverflow.com/questions/54450398/how-to-fix-gradle-task-appmergedebugjnilibfolders-in-flutter-gradle-build-fo', 'https://stackoverflow.com/questions/69431521/flutter-android-build-error-in-azure-pipeline-no-toolchains-found-in-the-ndk-to', 'https://stackoverflow.com/questions/60475481/flutter-doctor-error-android-sdkmanager-tool-not-found-windows']


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.tools.retriever import create_retriever_tool


if links:
  
    loaders = [WebBaseLoader(link) for link in links]
    loaded_docs = [loader.load() for loader in loaders]

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = [text_splitter.split_documents(docs) for docs in loaded_docs]

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    retriever_tools = []
    for i, (link, docs) in enumerate(zip(links, split_docs)):
        vectordb = Chroma.from_documents(docs, embeddings)
        retriever = vectordb.as_retriever()

        tool = create_retriever_tool(
            retriever,
            f"stackoverflow_search_{i+1}",
            f"Search for solutions from Stack Overflow article: {link}",
        )
        retriever_tools.append(tool)
        print(f"Created retriever tool for {link}")

else:
    print("No Stack Overflow links found.")


C:\Users\PRIYANSHU\AppData\Local\Temp\ipykernel_1688\2720506034.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Created retriever tool for https://stackoverflow.com/questions/54450398/how-to-fix-gradle-task-appmergedebugjnilibfolders-in-flutter-gradle-build-fo
Created retriever tool for https://stackoverflow.com/questions/69431521/flutter-android-build-error-in-azure-pipeline-no-toolchains-found-in-the-ndk-to
Created retriever tool for https://stackoverflow.com/questions/60475481/flutter-doctor-error-android-sdkmanager-tool-not-found-windows


In [27]:
retriever_tools

[Tool(name='stackoverflow_search_1', description='Search for solutions from Stack Overflow article: https://stackoverflow.com/questions/54450398/how-to-fix-gradle-task-appmergedebugjnilibfolders-in-flutter-gradle-build-fo', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B199F7DC60>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B1B9E8FE80>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001B19A1A83A0>, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001B1B9E8FE80>, search

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

f:\Programming\AI - Ml\Langchain_Tutorial\Basic rag\myenv\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [30]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm,retriever_tools,prompt)

In [32]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=retriever_tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMe

In [44]:
response = agent_executor.invoke({"input": "No version of NDK matched the requested version"})
print(response)



> Entering new AgentExecutor chain...
This error often arises during Android development, particularly with Flutter projects.  It indicates that the Android NDK (Native Development Kit) version specified in your project doesn't match a version installed on your system. To troubleshoot, let's try to gather more information.  Could you share the following:

1. **The exact error message:**  Please provide the complete error message you're seeing.  The phrasing "No version of NDK matched the requested version" is generic, and the full message might contain crucial details like the requested NDK version.

2. **Your Flutter and Android Studio versions:** Knowing your versions will help determine compatibility issues.  Run `flutter --version` and check your Android Studio version in the "About" section.

3. **Your project's `local.properties` file (or equivalent):** This file specifies the NDK path.  Sharing its contents will reveal the NDK version your project expects.  (Be mindful of any 